In [26]:
import os
import gc
import re
import time
import random
import numpy as np
import pandas as pd
import csv
import json
import copy
import glob
import math
import joblib
import pickle
import torch
import ast
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.optim import AdamW
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from transformers import BertTokenizer, VisualBertConfig, VisualBertModel, AdamW

Dataset & Dataloader

In [8]:
from sklearn.model_selection import train_test_split
dataset_train = pd.read_csv("/Users/shengfang/Desktop/X.051/dataset_train.csv")

train_set, val_set = train_test_split(dataset_train, test_size=1/9, random_state=42, shuffle=True)
test_set = pd.read_csv("/Users/shengfang/Desktop/X.051/dataset_test.csv")

train_set = train_set.reset_index(drop=True)
val_set = val_set.reset_index(drop=True)

             jid formula_spaced            element_list crystal_system  \
0     JVASP-7706    Ga1 Li1 Si1      ['Ga', 'Li', 'Si']          cubic   
1    JVASP-39100  Ge1 N2 O1 Si1  ['Ge', 'N', 'O', 'Si']     monoclinic   
2    JVASP-12950    Na4 Se4 Sn1      ['Na', 'Se', 'Sn']     tetragonal   
3     JVASP-8119    Pt1 Sb1 Sc1      ['Pt', 'Sb', 'Sc']          cubic   
4      JVASP-113         O2 Zr1             ['O', 'Zr']     monoclinic   
..           ...            ...                     ...            ...   
543   JVASP-8665         Cl3 I1             ['Cl', 'I']      triclinic   
544  JVASP-10482     Cd1 Cu1 F4       ['Cd', 'Cu', 'F']     tetragonal   
545  JVASP-34311    C1 F1 H6 N1    ['C', 'F', 'H', 'N']       trigonal   
546  JVASP-46080     Dy2 O7 Pb2       ['Dy', 'O', 'Pb']          cubic   
547   JVASP-9813     Bi2 Mg1 O5       ['Bi', 'Mg', 'O']   orthorhombic   

     is_direct                                            en_list  \
0            0  [0.0, 0.015, 0.03, 0.045, 

In [28]:
class DataLoader():
    '''
    [Input]
    dataset_path:      Pre-loaded train/val/test dataset 
                       dataset: 'formula_spaced ', 'element_list', 'crystal_system', 'imag_avg', is_direct
                                'imag_avg': strings of Y. Y is dielectric function (a.u.), Y has constant length, covering energy(eV) range [0.,7.0,0.15]
    
    batch_size:        Batch size n.
    
    [Output]
    Xs:                Array of intensity in (n,Y,1), n = batch_size
    Ys:                Array of binary labels in (n, 0 or 1)
    formula_list:      Chemical formula for each sample
    element_list:      List of elements for each sample 
    sg_list:           Space group for each sample
    '''
    def __init__(self, dataset):
        
        self.dataset = dataset
        self.len = len(self.dataset)

    def load_data(self, batch_size = 16, shuffle=True):

        if shuffle:
            indices = np.random.permutation(self.len) 
        else:
            indices = np.arange(self.len)
        
        spectra_list = ast.literal_eval(self.dataset.iloc[0]['imag_avg'])
        spectra_len = len(spectra_list)
        
        self.Xs = np.zeros((batch_size,spectra_len,1))
        self.Ys = np.zeros((batch_size, 1))
        self.formula_list, self.element_list, self.sg_list = [], [], []

        i = 0
        while i < batch_size:
            idx = indices[i]
            sample = self.dataset.iloc[idx]
            
            formula = sample['formula_spaced']
            element_list = sample['element_list']
            sg_list = sample['crystal_system']
            spectra_list = ast.literal_eval(sample['imag_avg'])
            intensities = np.array(spectra_list)
            label = sample['is_direct']
            
            self.Xs[i, :, 0] = intensities/np.max(intensities)
            self.Ys[i] = label 

            self.formula_list.append(formula)
            self.element_list.append(element_list)
            self.sg_list.append(sg_list)

            i += 1

        return self.Xs, self.Ys, self.formula_list, self.element_list, self.sg_list

In [30]:
val_loader = DataLoader(val_set)
val_Xs, val_Ys, val_formula_list, val_element_list, val_sg_list = val_loader.load_data(batch_size = 3, shuffle=True)

fig, ax = plt.subplots(figsize=(12, 10))

for i in range(len(val_formula_list)):
    ax.plot(np.arange(0.00, 7.00, 0.015), val_Xs[i])
    print(val_formula_list[i], val_element_list[i], val_sg_list[i], val_Ys[i])
    
ax.xaxis.set_major_locator(mpl.ticker.MaxNLocator(nbins=7, steps=[1, 2, 5, 10]))
ax.xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(2))
ax.yaxis.set_major_locator(mpl.ticker.MaxNLocator(nbins=1, steps=[1, 2, 5, 10]))
ax.yaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(2))
ax.tick_params(axis='both',direction='in',length=8,width=2,pad=10,color='black',right='on',top='on',labelsize=28)
ax.tick_params(axis='both',which='minor',direction='in',length=4,width=2,pad=10,color='black',right='on',top='on',labelsize=28)
    
ax.axes.set_xlim([0.,7.])
ax.axes.set_ylim([-0.02,1.02])

ax.set_xlabel(r'Energy (eV)', labelpad=20, fontsize=34)
ax.set_ylabel(r'Normalized intensity', labelpad=0, fontsize=34)

Al1 B3 N4 ['Al', 'B', 'N'] cubic [1.]
Ag1 Pr5 Se8 ['Ag', 'Pr', 'Se'] tetragonal [0.]
Ca3 N2 ['Ca', 'N'] cubic [1.]


Text(0, 0.5, 'Normalized intensity')

In [32]:
train_loader = DataLoader(train_set)
val_loader = DataLoader(val_set)
test_loader = DataLoader(test_set)

In [34]:
tokenizer = BertTokenizer.from_pretrained("google-bert/bert-base-uncased")
visual_embedding_dim = 10
configuration = VisualBertConfig(
    vocab_size=len(tokenizer),
    visual_embedding_dim=visual_embedding_dim,
    hidden_size=int(467. / visual_embedding_dim),  # Ensure this is valid for your input shape
    num_attention_heads= 2,
    hidden_dropout_prob=0.2,
    num_labels=1,  # For binary classification (0 or 1)
)
print(configuration)
class VisualBertClassifier(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.visual_bert = VisualBertModel(config)
        self.classifier = nn.Linear(config.hidden_size, 1)  # Binary classification
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, visual_embeds=None, visual_attention_mask=None):
        outputs = self.visual_bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            visual_embeds=visual_embeds,
            visual_attention_mask=visual_attention_mask
        )
        pooled_output = outputs.pooler_output
        logits = self.classifier(pooled_output)
        return logits
        
# model = VisualBertClassifier(configuration).to('cuda')
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
model = VisualBertClassifier(configuration).to(device)  ### Running with Macbook
optimizer = AdamW(model.parameters(), lr=1e-4)

/Users/shengfang/anaconda3/envs/visualbert/lib/python3.12/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


VisualBertConfig {
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "bypass_transformer": false,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.2,
  "hidden_size": 46,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "visual_bert",
  "num_attention_heads": 2,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "special_visual_initialize": true,
  "transformers_version": "4.37.2",
  "type_vocab_size": 2,
  "visual_embedding_dim": 10,
  "vocab_size": 30522
}



/Users/shengfang/anaconda3/envs/visualbert/lib/python3.12/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
epochs = 100000
save_training_results = False
save_epochs = [5000,10000,20000,50000,100000]

model_folder = r'/Users/shengfang/Desktop/X.051'

train_losses,train_y_true,train_y_pred = [],[],[]
val_losses,val_y_true,val_y_pred = [],[],[]
for epoch in range(epochs):
    model.train()
    train_Xs, train_Ys, train_formula_list, train_element_list, train_sg_list = train_loader.load_data(batch_size = 20, shuffle=True)
    len_dim = (train_Xs.shape[1] // visual_embedding_dim) * visual_embedding_dim
    train_Xs = train_Xs[:, :len_dim,:]  # (20, 467, 1) → (20, 460, 1)
    train_Xs = train_Xs[:,:,:].reshape((train_Xs.shape[0],int(train_Xs.shape[1]/visual_embedding_dim),visual_embedding_dim))
    
    X_prompts = [f"{formula} {' '.join(elems)} {sg}" for formula, elems, sg in zip(train_formula_list, train_element_list, train_sg_list)]

    visual_embeds = torch.from_numpy(train_Xs).to(dtype=torch.float32)
    visual_attention_mask = torch.ones(visual_embeds.shape[:-1], dtype=torch.long)
    # visual_token_type_ids = torch.ones(visual_embeds.shape[:-1], dtype=torch.long)
            
    inputs = tokenizer(X_prompts, return_tensors="pt", padding=True)
    inputs.update(
        {
            "visual_embeds": visual_embeds,
            # "visual_token_type_ids": visual_token_type_ids,
            "visual_attention_mask": visual_attention_mask,
        }
    )
    
    labels = torch.from_numpy(np.array(train_Ys)).to(dtype=torch.float32)

    num_positives = np.sum(train_Ys)
    num_negatives = len(labels) - num_positives
    pos_weight_val = num_negatives / (num_positives + 1e-8)  # Avoid divide-by-zero
    pos_weight = torch.tensor([pos_weight_val], dtype=torch.float32).to(device)
    loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

    # outputs = model(**inputs.to('cuda'))
    logits = model(**inputs.to(device))
    # loss = loss_fn(logits, labels.to('cuda'))
    loss = loss_fn(logits, labels.to(device))  
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    y_true = np.array(labels)
    y_pred = torch.sigmoid(logits).detach().cpu().numpy()
    if save_training_results:
        train_y_true.append(y_true)
        train_y_pred.append(y_pred)
    
    if (epoch+1)%10 == 0:
        # print('{}\t{:.5f}\t{}'.format(epoch+1, np.array(loss.detach().cpu()), y_true, y_pred))
        print(f"Epoch {epoch+1}\tLoss: {loss.item():.5f}")
        print(y_true,y_pred)
    train_losses.append(np.array(loss.detach().cpu()))
    
    if (epoch+1) in save_epochs:
        # model.save_pretrained(os.path.join(model_folder, '{}'.format(epoch)), from_pt=True)
        torch.save(model.state_dict(), os.path.join(model_folder, f"{epoch}_model.pt"))
        np.savetxt(os.path.join(model_folder, 'train_loss.csv'), train_losses, delimiter=',')
        print('Save model for epoch: {}'.format(epoch))
        if save_training_results:
            with open(os.path.join(model_folder, 'train_result.npy'), 'wb') as handle:
                joblib.dump({'y_true':train_y_true, 'y_pred':train_y_pred, 'loss':train_losses}, handle)


    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():  # Disable gradients during validation
        val_Xs, val_Ys, val_formula_list, val_element_list, val_sg_list = val_loader.load_data(batch_size = 20, shuffle=True)
        len_dim = (val_Xs.shape[1] // visual_embedding_dim) * visual_embedding_dim
        val_Xs = val_Xs[:, :len_dim,:]  # (20, 467, 1) → (20, 460, 1)
        val_Xs = val_Xs[:,:,:].reshape((val_Xs.shape[0],int(val_Xs.shape[1]/visual_embedding_dim),visual_embedding_dim))
        # Prepare validation inputs
        X_prompts_val = [f"{formula} {' '.join(elems)} {sg}" for formula, elems, sg in zip(val_formula_list, val_element_list, val_sg_list)]

        val_visual_embeds = torch.from_numpy(val_Xs).to(dtype=torch.float32)
        val_attention_mask = torch.ones(val_visual_embeds.shape[:-1], dtype=torch.long)
        # val_token_type_ids = torch.ones(val_visual_embeds.shape[:-1], dtype=torch.long).to('cuda')
            
        val_inputs = tokenizer(X_prompts_val, return_tensors="pt", padding=True)
        val_inputs.update({
            "visual_embeds": val_visual_embeds,
            # "visual_token_type_ids": val_token_type_ids,
            "visual_attention_mask": val_attention_mask,
        })

        # Validation labels
        val_labels = torch.from_numpy(np.array(val_Ys)).to(dtype=torch.float32)

        num_positives = np.sum(val_Ys)
        num_negatives = len(labels) - num_positives
        pos_weight_val = num_negatives / (num_positives + 1e-8)  # Avoid divide-by-zero
        pos_weight = torch.tensor([pos_weight_val], dtype=torch.float32).to(device)
        loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
        # Forward pass for validation
        # val_outputs = model(**val_inputs.to('cuda'))
        val_outputs = model(**val_inputs.to(device))
        val_logits = val_outputs
        # loss = loss_fn(val_logits, val_labels.to('cuda'))
        val_loss = loss_fn(val_logits, val_labels.to(device))

        y_true_val = np.array(val_labels)
        y_pred_val = torch.sigmoid(val_logits).detach().cpu().numpy()
        if save_training_results:
            val_y_true.append(y_true_val)
            val_y_pred.append(y_pred_val)
        if (epoch+1)%10 == 0:
            # print('{}\t{:.5f}\t{}'.format(epoch+1, np.array(val_loss.detach().cpu())))
            print(f"Epoch {epoch+1}\tLoss: {val_loss.item():.5f}")
        val_losses.append(np.array(val_loss.detach().cpu()))

    if (epoch+1) in save_epochs: 
        np.savetxt(os.path.join(model_folder, 'val_loss.csv'), val_losses, delimiter=',')

Epoch 10	Loss: 1.02676
[[1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]] [[0.5057835 ]
 [0.507194  ]
 [0.49951982]
 [0.50317806]
 [0.48923072]
 [0.49127632]
 [0.49935853]
 [0.4921807 ]
 [0.517587  ]
 [0.46942723]
 [0.47719407]
 [0.50958806]
 [0.4826802 ]
 [0.49275038]
 [0.49498367]
 [0.50619835]
 [0.5060548 ]
 [0.49586064]
 [0.48759288]
 [0.5121761 ]]
Epoch 10	Loss: 1.31751
Epoch 20	Loss: 0.82693
[[0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]] [[0.5078199 ]
 [0.49202466]
 [0.52415174]
 [0.49219596]
 [0.5046892 ]
 [0.48424417]
 [0.5038313 ]
 [0.52155286]
 [0.49674535]
 [0.51135474]
 [0.53934014]
 [0.4945218 ]
 [0.51281655]
 [0.49714813]
 [0.49522528]
 [0.4961968 ]
 [0.5043982 ]
 [0.50057083]
 [0.5206837 ]
 [0.49065977]]
Epoch 20	Loss: 1.10828
Epoch 30	Loss: 1.03272
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0

In [ ]:
model.load_state_dict(torch.load("##load model##"))
model.to(device)
model.eval()
test_y_true,test_y_pred= [],[]
with torch.no_grad():
    test_Xs, test_Ys, test_formula_list, test_element_list, test_sg_list = test_loader.load_data(batch_size=len(test_loader.dataset), shuffle=False)

    len_dim = (test_Xs.shape[1] // visual_embedding_dim) * visual_embedding_dim
    test_Xs = test_Xs[:, :len_dim, :]
    test_Xs = test_Xs.reshape((test_Xs.shape[0], -1, visual_embedding_dim))

    test_prompts = [f"{formula} {' '.join(elems)} {sg}" for formula, elems, sg in zip(test_formula_list, test_element_list, test_sg_list)]

    test_visual_embeds = torch.from_numpy(test_Xs).to(dtype=torch.float32)
    test_attention_mask = torch.ones(test_visual_embeds.shape[:-1], dtype=torch.long)

    test_inputs = tokenizer(test_prompts, return_tensors="pt", padding=True)
    test_inputs = {k: v.to(device) for k, v in test_inputs.items()}
    test_inputs.update({
        "visual_embeds": test_visual_embeds,
        "visual_attention_mask": test_attention_mask,
    })

    test_labels = torch.from_numpy(np.array(test_Ys)).to(dtype=torch.float32)

    test_logits = model(**test_inputs)
    test_loss = loss_fn(test_logits, test_labels)

    y_true_test = test_labels.cpu().numpy()
    y_pred_test = torch.sigmoid(test_logits).detach().cpu().numpy()
    if y_pred_test > 0.5:
        y_pred_test = 1.
    else:
        y_pred_test = 0.

    test_y_true.append(y_true_test)
    test_y_pred.append(y_pred_test)

print(test_y_true, test_y_pred)